## **Librerías**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.options.display.max_columns = False

In [3]:
from datetime import datetime, timedelta
import winsound as cst

In [4]:
import sys
sys.path.append('./')
sys.path.append('../')

In [6]:
import xml.etree.ElementTree as ET

## **Variables Globales**

In [5]:
dia = int(str(datetime.now()).replace('-', '')[:8])
mes = int(str(datetime.now()).replace('-', '')[:6])

## **Datos**

In [7]:
path = '../Data/Raw/feed.xml'

In [8]:
# Cargar el archivo XML
tree = ET.parse(path)
root = tree.getroot()

In [9]:
data = []
for person in root.findall("person"):
    data.append({
        "name": person.get("name"),
        "age": int(person.get("age")),
        "role": person.text.strip()
    })

In [ ]:
df_raw = pd.DataFrame(data)

[]

## **Limpieza**

## **Analísis y Exploración**